Le but de ce notebook est de comparer la performance de nos algorithmes (basée sur la métrique RMSE) en fonction de la taille de l'entrée lorsqu'on prend des matrices aléatoires.

# Importation des modules nécessaires

In [3]:
import numpy as np
import numpy.linalg as alg
from utilities import generate_low_rank, generate_entries
from convex_optimization import test_convex
from gradient_descent_script import generate_prediction_noprint, generate_M
from clustering_method import complete_matrix
from clean_altmin import predict_AltMin
import matplotlib.pyplot as plt
from tqdm import tqdm

In [4]:
import warnings
warnings.filterwarnings("ignore")

# Variables et fonctions pour tester chaque algorithme

In [1]:
N=1000 #taille maximale
step=50 #pas sur les tailles
nb_iter=1 #nombre d'expériences pour chaque valeur de la taille

In [6]:
test_gradient=lambda A,omega: alg.norm(A-generate_prediction_noprint(A,omega,learning_rate=0.1))/A.size**0.5
def test_alternating(A,omega):
    M=generate_M(A,omega)
    P=predict_AltMin(M,*A.shape)
    return alg.norm(A-P)/P.size**0.5
def test_clustering(A,omega): 
    n,p=A.shape
    M=np.zeros((n,p))
    for i,j in omega:
        M[i,j]=A[i,j]
    return alg.norm(A-complete_matrix(M))/A.size**0.5

Chacune de ces fonctions prend en entrée la matrice complète et l'ensemble omega des entrées révélées et retourne la RMSE de la prédiction de l'algorithme correspondant.

# Traçage des courbes

In [ ]:
n_list=np.arange(50,N,step)
RMSE_convex=np.zeros_like(n_list)
RMSE_gradient=np.zeros_like(n_list)
RMSE_alternating=np.zeros_like(n_list)
RMSE_clustering=np.zeros_like(n_list)
for p in tqdm(range(n_list.shape)):
    n=n_list[p]
    X_convex=np.ndarray(nb_iter)
    X_gradient=np.ndarray(nb_iter)
    X_alternating=np.ndarray(nb_iter)
    X_clustering=np.ndarray(nb_iter)
    for i in range(nb_iter):
        omega=generate_entries(n,n,int(5*n*np.log(n)))
        A=generate_low_rank(n,n,int(n**(1/5)))
        X_convex[i]=test_convex(A,omega)
        X_gradient[i]=test_gradient(A,omega)
        X_clustering[i]=test_clustering(A,omega)
        X_alternating[i]=test_alternating(A,omega)
    RMSE_convex[p]=np.mean(X_convex)
    RMSE_gradient[p]=np.mean(X_gradient)
    RMSE_alternating[p]=np.mean(X_alternating)
    RMSE_clustering[p]=np.mean(X_clustering)

In [ ]:
plt.plot(n_list,RMSE_convex,label="Optimisation convexe")
plt.plot(n_list,RMSE_gradient,label="Descente du gradient")
plt.plot(n_list,RMSE_alternating,label="Minimisation alternée")
plt.plot(n_list,RMSE_clustering,label="Clustering")
plt.ylabel("RMSE moyen")
plt.xlabel("taille de la matrice")
plt.title("RMSE en fonction de la taille")
plt.legend()
plt.show()